In [59]:
import subprocess
import os
from pathlib import Path
import shutil
import sys
import re
from enum import Enum 
import random
import copy
import time

gradle_dir = '/Users/adityachawla/bc22/Battlecode22'
home_dir = '/Users/adityachawla/bc22/Battlecode22/src'

os.chdir(home_dir)

target_bot = 'defensivebot2'

In [60]:
MUTATION_RATE = 10

In [61]:
def iter_dir(to_iter):
    for root, subdirs, files in os.walk(to_iter):
        for file in files:
            if file.endswith('.java'):
                yield root,os.path.join(root,file)

def delete_dir(to_del):
    name_path = Path(to_del)
    if name_path.exists():
        shutil.rmtree(name_path)
            
def copy_bot(from_bot,to_bot):
    delete_dir(to_bot)

    for root,file in iter_dir(from_bot):
        n_root = root.replace(from_bot,to_bot)
        n_file = file.replace(from_bot,to_bot)
        Path(n_root).mkdir(parents=True, exist_ok=True)
        out = Path(n_file)
        Path.touch(out)
        with open(file, 'r') as file:
            content = file.read()
        out_content = re.sub(from_bot, to_bot, content)
        with open(out, 'w') as out_file:
            out_file.write(out_content)
            
def double_arr_parser(line):
    vals = re.sub('[{};]','',line).split(',')
    return [float(val.strip()) for val in vals]

def int_parser(line):
    return [int(line.strip().replace(";",""))]

def double_parser(line):
    return [float(line.strip().replace(";",""))]

def number_builder(vals):
    return str(vals[0])+';';

def number_array_rebuilder(vals):
    return '{'+','.join(str(x) for x in vals)+'};'


variable_types = {
    'double_arr':{
    'type':'double[]',
    'parser' : double_arr_parser,
    'rebuilder' : number_array_rebuilder
    },
    'int':{
    'type':'int',
    'parser' : int_parser,
    'rebuilder' : number_builder
    },
    'double':{
    'type':'double',
    'parser' : double_parser,
    'rebuilder' : number_builder
    },
}

class Variable:
    def __init__(self,name:str,var_type,min_vals,max_vals,steps):
        self.name = name
        self.var_type = var_type
        self.mins = min_vals
        self.maxs = max_vals
        self.steps = steps
        self.vals = None
        
    def set_values(self,vals):
        self.vals = vals

class Conf:
    def __init__(self,path:str,variables):
        self.path = path
        self.variables = variables
    
    def copy():
        return Conf(path,[variable.copy()])
        
class Bot:
    def __init__(self,name,confs,link=False):
        self.name = name
        self.confs = confs
        self.fitness = 0
        if link:
            self.link_local()
        
    def link_local(self):
        for conf in self.confs:
            read_conf(self.name,conf)
    
    def __repr__(self):
        return f'bot {self.name}, fitness:{self.fitness}'
    
    
def read_conf(bot,conf):
    file = os.path.join(bot,conf.path)
    with open(file,'r') as f:
        content = f.readlines()
    
    for line in content:
        for variable in conf.variables:
            var_type = variable_types[variable.var_type]
            to_find = var_type['type'] + ' ' + variable.name
            if to_find in line:
                left,right = line.split('=')
                vals = re.sub(r'[{};]','',right.strip())
                variable.set_values(var_type['parser'](vals))
                break

def set_conf(bot,conf):
    file = os.path.join(bot,conf.path)
    with open(file,'r') as f:
        content = f.readlines()

    new_content = ''

    for i,line in enumerate(content):
        content_added = False
        for variable in conf.variables:
            var_type = var_type = variable_types[variable.var_type]
            to_find = var_type['type'] + ' ' + variable.name
            if to_find in line:
                left,right = line.split('=')
                n_vals = var_type['rebuilder'](variable.vals)
                new_content += left + '=' + n_vals + '\n'
                content_added = True
                break
        if not content_added:
            new_content += line
    os.remove(file)

    with open(file, 'w') as out_file:
        out_file.write(new_content)
        
def match(teamA='examplefuncsplayer',teamB='examplefuncsplayer',field='eckleburg'):
    s = f"{gradle_dir}/gradlew run -PteamA={teamA} -PteamB={teamB} -Pmaps={field} -PprofilerEnabled=false"
    return s.split()

def run_matches(bot_a,bot_b,maps=['eckleburg']):
    return subprocess.Popen(match(bot_a,bot_b,','.join(maps)), stdout=subprocess.PIPE,cwd=gradle_dir) 

def get_results_from_process(process):
    logs_str = process.stdout.read().decode("utf-8") 
    logs = logs_str.split('\n')
    
    found_vs = False
    found_win = False
    results = []
    for log in logs:
        vs = re.findall(r'([a-zA-Z0-9]*) vs. ([a-zA-Z0-9]*) on ([a-zA-Z0-9]*)',log)
        win = re.findall(r'([a-zA-Z0-9]*) \([AB]\) wins \(round (\d*)\)',log)

        if len(vs) == 1:
            found_vs = True
            team_a,team_b,field = vs[0]

        if len(win) == 1:
            found_win = True
            winner, rounds = win[0][0],int(win[0][1])

        if found_vs and found_win:
            found_vs = found_win = False

            loser = team_a
            if winner == team_a:
                loser = team_b
            
            print(winner,'won in rounds',rounds)
            result = MatchResult(winner,loser,field,rounds)
            results.append(result)
    return results

In [62]:
def chance(prob):
    r=random.randint(0,99)
    return r<prob

def choose(bots,exclude_bot):
    if len(bots) == 1 and exclude_bot is not None:
        return None
        
    fit_sum = sum([bot.fitness for bot in bots if bot != exclude_bot])
    r = random.randint(0,int(fit_sum))
    s = 0
    for bot in bots:
        if bot == exclude_bot:
            continue
        s += bot.fitness
        if s>=r:
            print('choosing',bot,'as parent')
            return bot
    return random.choice(bots)

def mutate(min_val,max_val,step):
    granularity = int((max_val - min_val)/step)
    val = min_val + round(random.randint(0,granularity)*step,2)
    return val

def mate(a,b,name,avg_mating_chance=50,mutation_rate=3):
    n_confs = []
    for conf_a,conf_b in zip(a.confs,b.confs):
        assert conf_a.path == conf_b.path
        conf_c = copy.deepcopy(conf_a)
        for var_a,var_b,var_c in zip(conf_a.variables,conf_b.variables,conf_c.variables):
            assert var_b.name == var_a.name
            assert var_b.var_type == var_a.var_type
            
            # mate two variables
            c_vals = []
            
            for i in range(len(var_a.vals)):
                val_a = var_a.vals[i]
                val_b = var_b.vals[i]
            
            for val_a,val_b in zip(var_a.vals,var_b.vals):
                if chance(mutation_rate):
                    # mutate variable
                    val_c = mutate(var_a.mins[i],var_a.maxs[i],var_b.steps[i])
                    
                elif chance(avg_mating_chance):
                    # mate by average
                    if var_a.var_type.startswith('int'):
                        val_c = (val_a+val_b)//2
                    else:
                        val_c = (val_a+val_b)/2
                else:
                    if chance(50):
                        # use value of a
                        val_c = val_a
                    else:
                        # use value of b
                        val_c = val_b
                c_vals.append(round(val_c,2))
            var_c.set_values(c_vals)
        n_confs.append(conf_c)
    return Bot(name,n_confs)

class MatchResult:
    def __init__(self,winner,loser,field,rounds):
        self.winner=winner
        self.loser=loser
        self.field=field
        self.rounds=rounds
        
    def __repr__(self):
        return f'winner:{self.winner}, loser:{self.loser}, field:{self.field}, rounds:{self.rounds}'

In [63]:
class Generation:
    def __init__(self,bots,ref_bot = 'defensivebot2'):
        self.named_bots = {bot.name:bot for bot in bots}
        self.bots = bots
        self.results = []
        self.done = False
        
        for bot in self.bots:
            copy_bot(ref_bot,bot.name)
            
            for conf in bot.confs:
                set_conf(bot.name,conf)
    
    def init_bots(self,bots,ref_bot='defensivebot2'):
        self.bots = bots
        
        for bot in self.bots:
            copy_bot(ref_bot,bot.name)
            
            for conf in bot.confs:
                set_conf(bot.name,conf)
        
    def play_matches(self,fields):
        n = len(self.bots)
#         processes = []
        for i in range(n):
            for j in range(i+1,n):
                print(self.bots[i],'fighting with',self.bots[j])
                process = run_matches(self.bots[i].name,self.bots[j].name,fields)
                self.results.extend(get_results_from_process(process))
                process_rev = run_matches(self.bots[j].name,self.bots[i].name,fields)
                self.results.extend(get_results_from_process(process_rev))
#                 processes.append(process)
#                 processes.append(process_rev)
        
#         print(len(processes),'total processes')
#         for i,process in enumerate(processes):
#             print('getting result of process',i)
            
        self.done = True
    
    def generate_next_gen(self):
        if not self.done:
            print('play out the matches first')
            return None
        for bot in self.bots:
            bot.fitness = 0
            
        for result in self.results:
            self.named_bots[result.winner].fitness += 10000/result.rounds
        
        print('current gen',self.named_bots)
        
        n_bots = []
        i = 0
        while len(n_bots)<len(self.bots):
            pa = choose(self.bots,None)
            pb = choose(self.bots,pa)
            n_bots.append(mate(pa,pb,f'gabot{i}',50,MUTATION_RATE))
            i += 1
        return Generation(n_bots)
    
def get_random_gen_from_bot(bot,size,confs):
    bots = []
    ref_bot = Bot(bot,confs)
    for i in range(size):
        bots.append(mate(ref_bot,ref_bot,f'gabot{i}',0,50))
    return Generation(bots)

In [64]:
archon_path = 'bots/Archon.java'
soldier_weights = Variable(
    'soldierWeights','double_arr',
    [-10 for _ in range(9)],[10 for _ in range(9)],
    [0.01 for _ in range(9)]
)

miner_weights = Variable(
    'minerWeights','double_arr',
    [-10 for _ in range(9)],[10 for _ in range(9)],
    [0.01 for _ in range(9)]
)

phase_two_weights = Variable(
    'phaseTwoWeights','double_arr',
    [-10 for _ in range(7)],[10 for _ in range(7)],
    [0.01 for _ in range(7)]
)

random_mark_unexplored = Variable(
    'randomMarkUnexplored','int',
    [1],[100],[1]
)

archon_variables = [soldier_weights,miner_weights,phase_two_weights,random_mark_unexplored]
archon_conf = Conf(archon_path,archon_variables)

constants_path = 'utils/Constants.java'
lead_lower_thresh = Variable(
    'LEAD_LOWER_THRESHOLD_FOR_SENSING','int',
    [5],[17],[1]
)

lead_upper_thresh = Variable(
    'LEAD_UPPER_THRESHOLD_FOR_SENSING','int',
    [17],[30],[1]
)

mines_per_round = Variable(
    'MINES_PER_ROUND','double',
    [2.0],[20.0],[0.5]
)

close_radius = Variable(
    'CLOSE_RADIUS','int',
    [2],[10],[1]
)

rounds_per_action = Variable(
    'ROUNDS_PER_ACTION','int',
    [1],[5],[1]
)


dense_comms_update_limit = Variable(
    'DENSE_COMMS_UPDATE_LIMIT','int',
    [1],[6],[1]
)

archon_death_confirmation = Variable(
    'ARCHON_DEATH_CONFIRMATION','int',
    [1],[8],[1]
)

rounds_before_charge = Variable(
    'RUN_ROUNDS_BEFORE_CHARGE','int',
    [1],[20],[1]
)

builder_watch_tower_fraction = Variable(
    'BUILDER_WATCHTOWER_FRACTION','double',
    [0.1],[1],[0.01]
)

robots_sensing_threshold = Variable(
    'ROBOTS_UPPER_THRESHOLD_FOR_SENSING','int',
    [5],[20],[1]
)

builder_inch_forward = Variable(
    'BUILDER_INCH_FORWARD','int',
    [5],[20],[1]
)

archon_close_radius = Variable(
    'LEAD_MOVE_THRESHOLD','int',
    [20],[150],[1]
)

lead_move_thresh = Variable(
    'ARCHON_CLOSE_RADIUS','int',
    [20],[150],[1]
)

lead_worth_pursuing = Variable(
    'LEAD_WORTH_PURSUING','int',
    [1],[30],[1]
)

builder_lead_thresh = Variable(
    'BUILDER_LEAD_THRESH','int',
    [150],[1500],[10]
)

choose_sector_ga = Variable(
    'CHOOSE_SECTOR_GA','int',
    [1],[10],[1]
)

constants_variables = [
    lead_lower_thresh,lead_upper_thresh,mines_per_round,close_radius,rounds_per_action,
    dense_comms_update_limit,archon_death_confirmation,builder_watch_tower_fraction,
    rounds_before_charge,robots_sensing_threshold,builder_inch_forward,archon_close_radius,
    lead_move_thresh,lead_worth_pursuing,builder_lead_thresh,choose_sector_ga
]

constants_conf = Conf(constants_path,constants_variables)


pathfinding_path = 'utils/PathFindingConstants.java'

rubble_score_mult = Variable(
    'RUBBLE_SCORE_MULTIPLIER','double',
    [1],[15],[0.1]
)

rubble_score_local_mult = Variable(
    'RUBBLE_SCORE_LOCAL_MULTIPLIER','double',
    [1],[15],[0.1]
)

min_path_otp = Variable(
    'MIN_AFFORD_PATH_OPT','int',
    [1500],[4000],[20]
)

num_better_path = Variable(
    'NUM_BETTER_LOC_ITERATIONS','int',
    [1],[8],[1]
)

soldier_pathfiding_limit = Variable(
    'SOLDIER_PATHFINDING_LIMIT','int',
    [1500],[4000],[20]
)

miner_path_limit = Variable(
    'MINER_PATHFINDING_LIMIT','int',
    [1500],[4000],[20]
)

default_limit = Variable(
    'DEFAULT_LIMIT','int',
    [1500],[4000],[20]
)

caching_rubble_limit = Variable(
    'CACHING_RUBBLE_LIMIT','int',
    [5],[40],[1]
)

vision_rad = Variable(
    'VISION_RADIUS','int',
    [3],[15],[1]
)

vision_rad_bias = Variable(
    'VISION_BIAS','int',
    [-5],[5],[1]
)


pathfinding_variables = [
    rubble_score_mult,rubble_score_local_mult,min_path_otp,num_better_path,soldier_pathfiding_limit,
    miner_path_limit,default_limit,caching_rubble_limit,vision_rad,vision_rad_bias
]
pathfinding_conf = Conf(pathfinding_path,pathfinding_variables)

In [65]:
ref_bot = Bot(target_bot,[archon_conf,constants_conf,pathfinding_conf],True)

In [80]:
read_conf(target_bot,archon_conf)
read_conf(target_bot,constants_conf)
read_conf(target_bot,pathfinding_conf)

gen_zero = get_random_gen_from_bot(ref_bot,4,[archon_conf,constants_conf,pathfinding_conf])

In [81]:
maps = "chessboard,collaboration,colosseum,dodgeball,eckleburg,equals,fortress,highway,intersection,jellyfish,maptestsmall,nottestsmall,nyancat,panda,pillars,progress,rivers,sandwich,snowflake,spine,squer,stronghold,tower,uncomfortable,underground,valley".split(",")


In [ ]:
# gen_zero.init_bots(gen.bots)
# gen = gen_zero
MUTATION_RATE = 5
for i in range(100):
    if i % 2 == 0:
        matches = random.choices(maps,k=3)
    gen.play_matches(matches)
    gen = gen.generate_next_gen()

bot gabot0, fitness:0 fighting with bot gabot1, fitness:0


Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot1 won in rounds 878
gabot1 won in rounds 904
gabot1 won in rounds 1406
gabot1 won in rounds 2000
gabot0 won in rounds 2000
gabot1 won in rounds 1021
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 775
gabot2 won in rounds 373
gabot2 won in rounds 1222
gabot2 won in rounds 1165
gabot2 won in rounds 781
gabot2 won in rounds 859
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot3 won in rounds 572
gabot3 won in rounds 364
gabot3 won in rounds 2000
gabot3 won in rounds 860
gabot3 won in rounds 262
gabot3 won in rounds 882
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot4 won in rounds 1432
gabot4 won in rounds 542
gabot4 won in rounds 2000
gabot4 won in rounds 1061
gabot4 won in rounds 261
gabot4 won in rounds 835
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 803
gabot2 won in rounds 309
gabot2 won in rounds 1431
gabot1 won in rounds 777
gabot2 won in rounds 477
gabot1 won in rounds 1752
bot ga

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot1 won in rounds 776
gabot1 won in rounds 599
gabot1 won in rounds 2000
gabot0 won in rounds 688
gabot0 won in rounds 461
gabot1 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot0 won in rounds 1312
gabot0 won in rounds 894
gabot2 won in rounds 2000
gabot0 won in rounds 664
gabot0 won in rounds 479
gabot0 won in rounds 1442
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot3 won in rounds 681
gabot3 won in rounds 440
gabot0 won in rounds 2000
gabot0 won in rounds 1052
gabot0 won in rounds 420
gabot3 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot0 won in rounds 788
gabot4 won in rounds 724
gabot4 won in rounds 2000
gabot0 won in rounds 873
gabot0 won in rounds 439
gabot4 won in rounds 2000
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 1628
gabot2 won in rounds 298
gabot2 won in rounds 2000
gabot1 won in rounds 863
gabot1 won in rounds 540
gabot1 won in rounds 2000
bot g

...

gabot1 won in rounds 2000
gabot1 won in rounds 593
gabot0 won in rounds 911
gabot0 won in rounds 1076
gabot0 won in rounds 414
gabot0 won in rounds 1607
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 653
gabot2 won in rounds 672
gabot0 won in rounds 520
gabot2 won in rounds 2000
gabot2 won in rounds 447
gabot0 won in rounds 1294
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot3 won in rounds 971
gabot0 won in rounds 749
gabot0 won in rounds 518
gabot3 won in rounds 1219
gabot0 won in rounds 321
gabot3 won in rounds 1635
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot0 won in rounds 989
gabot4 won in rounds 950
gabot4 won in rounds 2000
gabot0 won in rounds 2000
gabot4 won in rounds 483
gabot4 won in rounds 797
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot1 won in rounds 2000
gabot1 won in rounds 643
gabot2 won in rounds 642
gabot2 won in rounds 1905
gabot1 won in rounds 629
gabot1 won in rounds 826
bot gab

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot1 won in rounds 2000
gabot0 won in rounds 2000
gabot1 won in rounds 1602
gabot1 won in rounds 1664
gabot0 won in rounds 2000
gabot1 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 1408
gabot2 won in rounds 2000
gabot2 won in rounds 2000
gabot2 won in rounds 2000
gabot2 won in rounds 2000
gabot0 won in rounds 1107
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot0 won in rounds 1462
gabot0 won in rounds 1916
gabot3 won in rounds 1743
gabot3 won in rounds 1713
gabot0 won in rounds 1026
gabot3 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot4 won in rounds 1439
gabot0 won in rounds 2000
gabot0 won in rounds 2000
gabot0 won in rounds 2000
gabot0 won in rounds 2000
gabot0 won in rounds 1382
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot1 won in rounds 1536
gabot1 won in rounds 1302
gabot2 won in rounds 1264
gabot2 won in rounds 2000
gabot1 won in rounds 2000
gabot1 won in 

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot0 won in rounds 2000
gabot1 won in rounds 2000
gabot0 won in rounds 1782
gabot0 won in rounds 2000
gabot0 won in rounds 2000
gabot0 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot0 won in rounds 2000
gabot0 won in rounds 2000
gabot2 won in rounds 728
gabot2 won in rounds 2000
gabot0 won in rounds 764
gabot0 won in rounds 1047
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot0 won in rounds 2000
gabot0 won in rounds 1494
gabot3 won in rounds 922
gabot3 won in rounds 2000
gabot3 won in rounds 2000
gabot3 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot0 won in rounds 2000
gabot4 won in rounds 1184
gabot4 won in rounds 1630
gabot4 won in rounds 2000
gabot4 won in rounds 1219
gabot4 won in rounds 1601
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 1578
gabot1 won in rounds 2000
gabot1 won in rounds 2000
gabot2 won in rounds 2000
gabot1 won in rounds 2000
gabot2 won in rou

In [420]:
for i in range(8):
    matches = ['progress','rivers','valley']
    
    gen.play_matches(matches)
    gen = gen.generate_next_gen()

bot gabot0, fitness:0 fighting with bot gabot1, fitness:0


Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot0 won in rounds 336
gabot0 won in rounds 494
gabot1 won in rounds 493
gabot1 won in rounds 369
gabot1 won in rounds 585
gabot0 won in rounds 509
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot0 won in rounds 336
gabot0 won in rounds 494
gabot2 won in rounds 497
gabot2 won in rounds 368
gabot2 won in rounds 604
gabot0 won in rounds 500
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot0 won in rounds 336
gabot0 won in rounds 494
gabot3 won in rounds 493
gabot3 won in rounds 369
gabot3 won in rounds 585
gabot0 won in rounds 509
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot0 won in rounds 2000
gabot0 won in rounds 494
gabot4 won in rounds 538
gabot4 won in rounds 2000
gabot0 won in rounds 650
gabot0 won in rounds 586
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot1 won in rounds 341
gabot2 won in rounds 559
gabot2 won in rounds 502
gabot2 won in rounds 336
gabot1 won in rounds 558
gabot1 won in rounds 501
bot gabot1, fitn

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot0 won in rounds 2000
gabot1 won in rounds 473
gabot1 won in rounds 663
gabot1 won in rounds 2000
gabot0 won in rounds 1070
gabot1 won in rounds 548
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot0 won in rounds 2000
gabot0 won in rounds 839
gabot2 won in rounds 613
gabot2 won in rounds 2000
gabot0 won in rounds 769
gabot2 won in rounds 852
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot0 won in rounds 336
gabot3 won in rounds 559
gabot0 won in rounds 717
gabot3 won in rounds 414
gabot0 won in rounds 557
gabot0 won in rounds 514
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot0 won in rounds 336
gabot4 won in rounds 560
gabot4 won in rounds 504
gabot4 won in rounds 336
gabot0 won in rounds 558
gabot0 won in rounds 504
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot1 won in rounds 1333
gabot2 won in rounds 2000
gabot2 won in rounds 615
gabot2 won in rounds 1260
gabot2 won in rounds 2000
gabot1 won in rounds 2000
bot gabo

...

gabot0 won in rounds 402
gabot1 won in rounds 640
gabot1 won in rounds 517
gabot1 won in rounds 318
gabot0 won in rounds 723
gabot1 won in rounds 601
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot0 won in rounds 350
gabot2 won in rounds 990
gabot0 won in rounds 736
gabot2 won in rounds 323
gabot0 won in rounds 1054
gabot0 won in rounds 642
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot0 won in rounds 402
gabot3 won in rounds 640
gabot3 won in rounds 517
gabot3 won in rounds 318
gabot0 won in rounds 723
gabot3 won in rounds 601
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot4 won in rounds 644
gabot0 won in rounds 807
gabot4 won in rounds 966
gabot4 won in rounds 827
gabot0 won in rounds 668
gabot0 won in rounds 737
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 718
gabot2 won in rounds 820
gabot2 won in rounds 524
gabot1 won in rounds 809
gabot2 won in rounds 975
gabot1 won in rounds 509
bot gabot1, fitne

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot1 won in rounds 242
gabot1 won in rounds 594
gabot1 won in rounds 461
gabot1 won in rounds 272
gabot1 won in rounds 761
gabot1 won in rounds 793
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot2 won in rounds 242
gabot2 won in rounds 565
gabot2 won in rounds 461
gabot0 won in rounds 811
gabot0 won in rounds 795
gabot2 won in rounds 830
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot0 won in rounds 612
gabot3 won in rounds 517
gabot3 won in rounds 461
gabot3 won in rounds 273
gabot0 won in rounds 775
gabot3 won in rounds 793
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot0 won in rounds 2000
gabot4 won in rounds 814
gabot4 won in rounds 467
gabot4 won in rounds 272
gabot4 won in rounds 1431
gabot4 won in rounds 723
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot1 won in rounds 353
gabot2 won in rounds 697
gabot2 won in rounds 500
gabot2 won in rounds 353
gabot2 won in rounds 802
gabot1 won in rounds 500
bot gabot1, fitn

Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot0 won in rounds 251
gabot1 won in rounds 859
gabot0 won in rounds 833
gabot1 won in rounds 443
gabot0 won in rounds 536
gabot0 won in rounds 520
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot0 won in rounds 340
gabot0 won in rounds 697
gabot2 won in rounds 500
gabot2 won in rounds 340
gabot2 won in rounds 493
gabot0 won in rounds 500
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot0 won in rounds 2000
gabot3 won in rounds 581
gabot3 won in rounds 491
gabot3 won in rounds 743
gabot0 won in rounds 478
gabot0 won in rounds 492
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot4 won in rounds 281
gabot4 won in rounds 570
gabot4 won in rounds 496
gabot0 won in rounds 597
gabot4 won in rounds 668
gabot0 won in rounds 496
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot1 won in rounds 443
gabot2 won in rounds 628
gabot2 won in rounds 519
gabot2 won in rounds 251
gabot1 won in rounds 689
gabot2 won in rounds 779
bot gabot1, fitne

In [281]:
# bots = gen_zero.bots

In [403]:
gen.play_matches(['eckleburg','colosseum'])

bot gabot0, fitness:0 fighting with bot gabot1, fitness:0


Note: Some input files use unchecked or unsafe operations.
Note: Recompile with -Xlint:unchecked for details.


gabot0 won in rounds 622
gabot0 won in rounds 826
gabot1 won in rounds 646
gabot1 won in rounds 1051
bot gabot0, fitness:0 fighting with bot gabot2, fitness:0
gabot0 won in rounds 614
gabot0 won in rounds 2000
gabot2 won in rounds 956
gabot2 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot3, fitness:0
gabot3 won in rounds 767
gabot0 won in rounds 2000
gabot0 won in rounds 767
gabot3 won in rounds 2000
bot gabot0, fitness:0 fighting with bot gabot4, fitness:0
gabot4 won in rounds 984
gabot0 won in rounds 639
gabot4 won in rounds 592
gabot0 won in rounds 587
bot gabot1, fitness:0 fighting with bot gabot2, fitness:0
gabot1 won in rounds 619
gabot2 won in rounds 2000
gabot1 won in rounds 797
gabot1 won in rounds 926
bot gabot1, fitness:0 fighting with bot gabot3, fitness:0
gabot1 won in rounds 646
gabot1 won in rounds 1051
gabot3 won in rounds 672
gabot3 won in rounds 826
bot gabot1, fitness:0 fighting with bot gabot4, fitness:0
gabot4 won in rounds 678
gabot1 won in rounds

In [255]:
n_bots[0].confs[0].variables[0].vals

[-5.28, 1.0, 8.71, -1.0, 2.44, -1.0, 2.08, 0.01, -3.86]

In [406]:
from collections import Counter
Counter([r.winner for r in gen.results])

Counter({'gabot0': 8, 'gabot1': 9, 'gabot2': 6, 'gabot3': 9, 'gabot4': 8})